In [92]:
import subprocess
def get_html_from_url(url):
    cmds = ["/Applications/Google Chrome.app/Contents/MacOS/Google Chrome", "--headless", "--dump-dom", "--enable-features=ConversionMeasurement,AttributionReportingCrossAppWeb"]
    cmds.append(url)
    result = subprocess.run(cmds, stdout=subprocess.PIPE)
    text = result.stdout.decode('utf-8')
    return text
    
# t = get_html_from_url("https://www.khanacademy.org/math/algebra/x2f8bb11595b61c86:systems-of-equations/x2f8bb11595b61c86:introduction-to-systems-of-equations/v/trolls-tolls-and-systems-of-equation")
# t
# # result = subprocess.run(['pwd'], stdout=subprocess.PIPE)
# # text = result.stdout.decode('utf-8')
# # text


In [87]:
# unit_url = "https://www.khanacademy.org/math/algebra/x2f8bb11595b61c86:systems-of-equations/x2f8bb11595b61c86:introduction-to-systems-of-equations"
unit_url = "https://www.khanacademy.org/math/algebra/x2f8bb11595b61c86:functions"
site = "https://www.khanacademy.org"

In [93]:
from xml.dom.minidom import parseString

html = get_html_from_url(unit_url)



In [94]:
from html.parser import HTMLParser
class Parser(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self.href = ''
        self.urls = []

    def handle_starttag(self, tag, attrs):
        if (tag == 'a'):
            for (k,v) in attrs:
                if (k == 'href'):
                    self.href = v
        elif (tag == 'span'):
            for (k,v) in attrs:
                if (k == 'aria-label' and v == 'Video'):
                    self.urls.append(self.href)            



parser = Parser()
parser.feed(html)
parser.urls


['/math/algebra/x2f8bb11595b61c86:functions/x2f8bb11595b61c86:evaluating-functions/v/what-is-a-function',
 '/math/algebra/x2f8bb11595b61c86:functions/x2f8bb11595b61c86:evaluating-functions/v/understanding-function-notation-example-1',
 '/math/algebra/x2f8bb11595b61c86:functions/x2f8bb11595b61c86:evaluating-functions/v/understanding-function-notation-example-2',
 '/math/algebra/x2f8bb11595b61c86:functions/x2f8bb11595b61c86:evaluating-functions/v/evaluate-discrete-function',
 '/math/algebra/x2f8bb11595b61c86:functions/x2f8bb11595b61c86:evaluating-functions/v/evaluating-a-function-expression',
 '/math/algebra/x2f8bb11595b61c86:functions/x2f8bb11595b61c86:inputs-and-outputs-of-a-function/v/finding-input-given-function-output-formula',
 '/math/algebra/x2f8bb11595b61c86:functions/x2f8bb11595b61c86:inputs-and-outputs-of-a-function/v/matching-function-input-to-output-with-graph',
 '/math/algebra/x2f8bb11595b61c86:functions/x2f8bb11595b61c86:inputs-and-outputs-of-a-function/v/different-inputs-g

In [32]:
video_url = site + parser.urls[0]
video_html = get_html_from_url(video_url)




[0521/115618.386214:WARNING:runtime_features.cc(730)] AttributionReportingCrossAppWeb cannot be enabled in this configuration. Use --enable-features=ConversionMeasurement,AttributionReportingCrossAppWeb in addition.
[0521/115618.527860:WARNING:runtime_features.cc(730)] AttributionReportingCrossAppWeb cannot be enabled in this configuration. Use --enable-features=ConversionMeasurement,AttributionReportingCrossAppWeb in addition.


In [49]:
class VideoParser(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self.start_h1 = False
        self.start_div = False
        self.start_iframe = False
        
        self.name = ''
        self.desc = ''

    def handle_starttag(self, tag, attrs):
        if (tag == 'h1'):
            self.start_h1 = True       
        elif (tag == 'div'):
            for (k,v) in attrs:
                if (k == 'id' and v == 'ka-videoPageTabs-tabbedpanel-content'):
                    self.start_div = True
        elif (tag == 'iframe'):
            print(attrs)
            for (k,v) in attrs:
                if(k == 'data-youtubeid'):
                    print(v)

    def handle_endtag(self, tag):
        if (tag == 'h1'):
            self.start_h1 = False
        elif (tag == 'div'):
            self.start_div = False
            
    def handle_data(self, data):
        if self.start_h1:
            self.name = data.title()
        if self.start_div:
            self.desc += data

video_parser = VideoParser()
video_parser.feed(video_html)
print(video_parser.name, video_parser.desc)




[('src', 'https://www.googletagmanager.com/ns.html?id=GTM-WSX5PBP'), ('height', '0'), ('width', '0'), ('style', 'display:none;visibility:hidden')]
Systems Of Equations: Trolls, Tolls (1 Of 2) A troll forces us to use algebra to figure out the make-up of his currency. We end up setting up a system of equations. Created by Sal Khan.


In [78]:
def get_searched_value(video_html, searchStr):
    x = video_html.find(searchStr)
    start = video_html.find('"', x+len(searchStr))
    end = video_html.find('"', start+1)
    return video_html[start+1:end]


def get_topic(video_html):
    desc =  get_searched_value(video_html, '"translatedDescription":')
    name = get_searched_value(video_html, '"translatedTitle":').title()
    videoID = get_searched_value(video_html, '"translatedYoutubeId":')
    return {
        "name": name,
        "description": desc,
        "texts": ["text1.md"],
        "video_parts": [{
            "id": videoID
        }],
        "node_type": "lecture",
        "parent_id": "khan-algebra-1",
        "thumbnail": ""
    }

In [95]:

topics = []
for url in parser.urls:
    video_url = site + url
    video_html = get_html_from_url(video_url)
    topic = get_topic(video_html)
    print(json.dumps(topic, indent=4))
    topics.append(topic)
    

topics
    
    
    
    

{
    "name": "What Is A Function?",
    "description": "Functions assign a single output for each of their inputs. In this video, we see examples of various kinds of functions.",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "kvGsIo1TmsM"
        }
    ],
    "node_type": "lecture",
    "parent_id": "khan-algebra-1",
    "thumbnail": ""
}
{
    "name": "Worked Example: Evaluating Functions From Equation",
    "description": "Learn how to evaluate f(x)=49-x\u00b2 at x=5.",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "Id6UovYjd-M"
        }
    ],
    "node_type": "lecture",
    "parent_id": "khan-algebra-1",
    "thumbnail": ""
}
{
    "name": "Worked Example: Evaluating Functions From Graph",
    "description": "Evaluating a function at x=-1 using the graph of that function.",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "kzYtx_AqzjM"
        }
    ],


[0522/154541.540790:ERROR:shared_image_manager.cc(325)] SharedImageManager::ProduceMemory: Trying to Produce a Memory representation from a non-existent mailbox.


{
    "name": "Worked Example: Determining Domain Word Problem (All Integers)",
    "description": "Determining the domain of a function that models going up and down a ladder.",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "d1O54EQ1DSE"
        }
    ],
    "node_type": "lecture",
    "parent_id": "khan-algebra-1",
    "thumbnail": ""
}
{
    "name": "Recognizing Functions From Graph",
    "description": "Checking whether a given set of points can represent a function. For the set to represent a function, each domain element must have one corresponding range element at most.",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "qGmJ4F3b5W8"
        }
    ],
    "node_type": "lecture",
    "parent_id": "khan-algebra-1",
    "thumbnail": ""
}
{
    "name": "Does A Vertical Line Represent A Function?",
    "description": "Explaining why a vertical line doesn't represent a function.",
    "texts": [
      

[0522/154629.332448:ERROR:ssl_client_socket_impl.cc(879)] handshake failed; returned -1, SSL error code 1, net_error -100
[0522/154629.687110:ERROR:ssl_client_socket_impl.cc(879)] handshake failed; returned -1, SSL error code 1, net_error -100
[0522/154630.481329:ERROR:gles2_command_buffer_stub.cc(300)] ContextResult::kFatalFailure: Failed to create context.


{
    "name": "Recognizing Functions From Table",
    "description": "Checking whether a table of people and their heights can represent a function that assigns a height to a name.",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "5cK86VKoBPw"
        }
    ],
    "node_type": "lecture",
    "parent_id": "khan-algebra-1",
    "thumbnail": ""
}
{
    "name": "Recognizing Functions From Verbal Description",
    "description": "Checking whether y can be described as a function of x if y is always three more than twice x.",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "tBRW-CyJE6k"
        }
    ],
    "node_type": "lecture",
    "parent_id": "khan-algebra-1",
    "thumbnail": ""
}
{
    "name": "Recognizing Functions From Verbal Description Word Problem",
    "description": "Checking whether a description of the price of an order can be represented as a function of the shipping cost.",
    "texts": [
 

[0522/154843.344693:ERROR:ssl_client_socket_impl.cc(879)] handshake failed; returned -1, SSL error code 1, net_error -100
[0522/154843.568613:ERROR:ssl_client_socket_impl.cc(879)] handshake failed; returned -1, SSL error code 1, net_error -100
[0522/154844.314853:ERROR:gles2_command_buffer_stub.cc(300)] ContextResult::kFatalFailure: Failed to create context.


{
    "name": "Worked Example: Positive & Negative Intervals",
    "description": "Finding the positive or negative intervals of a function from its graph.",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "eDv7dk9uNmM"
        }
    ],
    "node_type": "lecture",
    "parent_id": "khan-algebra-1",
    "thumbnail": ""
}
{
    "name": "Graph Interpretation Word Problem: Temperature",
    "description": "When a function models a real-world context, we can learn a lot about the content from the function's graph. In this video, we consider a graph that models temperature over time.",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "VtBRkjRua0I"
        }
    ],
    "node_type": "lecture",
    "parent_id": "khan-algebra-1",
    "thumbnail": ""
}
{
    "name": "Graph Interpretation Word Problem: Basketball",
    "description": "When a function models a real-world context, we can learn a lot about the context 

[0522/155247.475886:ERROR:ssl_client_socket_impl.cc(879)] handshake failed; returned -1, SSL error code 1, net_error -100
[0522/155247.478444:ERROR:ssl_client_socket_impl.cc(879)] handshake failed; returned -1, SSL error code 1, net_error -100
[0522/155248.457198:ERROR:gles2_command_buffer_stub.cc(300)] ContextResult::kFatalFailure: Failed to create context.


{
    "name": "Graphing The Inverse Of A Linear Function",
    "description": "Sal is given a line segment on the coordinate plane, and he graphs the inverse of the function represented by that segment.",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "KPeS2gdizhQ"
        }
    ],
    "node_type": "lecture",
    "parent_id": "khan-algebra-1",
    "thumbnail": ""
}
{
    "name": "Finding Inverse Functions: Linear",
    "description": "Sal finds the inverses of f(x)=-x+4 and g(x)=-2x-1.",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "wSiamij_i_k"
        }
    ],
    "node_type": "lecture",
    "parent_id": "khan-algebra-1",
    "thumbnail": ""
}


[{'name': 'What Is A Function?',
  'description': 'Functions assign a single output for each of their inputs. In this video, we see examples of various kinds of functions.',
  'texts': ['text1.md'],
  'video_parts': [{'id': 'kvGsIo1TmsM'}],
  'node_type': 'lecture',
  'parent_id': 'khan-algebra-1',
  'thumbnail': ''},
 {'name': 'Worked Example: Evaluating Functions From Equation',
  'description': 'Learn how to evaluate f(x)=49-x² at x=5.',
  'texts': ['text1.md'],
  'video_parts': [{'id': 'Id6UovYjd-M'}],
  'node_type': 'lecture',
  'parent_id': 'khan-algebra-1',
  'thumbnail': ''},
 {'name': 'Worked Example: Evaluating Functions From Graph',
  'description': 'Evaluating a function at x=-1 using the graph of that function.',
  'texts': ['text1.md'],
  'video_parts': [{'id': 'kzYtx_AqzjM'}],
  'node_type': 'lecture',
  'parent_id': 'khan-algebra-1',
  'thumbnail': ''},
 {'name': 'Evaluating Discrete Functions',
  'description': 'Given the graph of a discrete function, Sal shows how to 